Assignment 2

In [28]:
import tensorflow as tf
from datasets import MNISTDataset
from time import time
import os

In [29]:
# get the data
(train_imgs, train_lbls), (test_imgs, test_lbls) = tf.keras.datasets.mnist.load_data()
mnist = MNISTDataset(train_imgs.reshape((-1, 784)), train_lbls,
                     test_imgs.reshape((-1, 784)), test_lbls,
                     batch_size=256, seed=int(time()))

In [30]:
#set up log dir and file writer(s)
logdir = os.path.join("logs", "linear" + str(time()))
train_writer = tf.summary.create_file_writer(os.path.join(logdir, "train"))
test_writer = tf.summary.create_file_writer(os.path.join(logdir, "test"))

In [31]:
# define the model first, from input to output

# let's use fewer layers...
n_units = 100
n_layers = 8 #8 #2


In [32]:
# just set up a "chain" of hidden layers
layers = []
for layer in range(n_layers):
    layers.append(tf.keras.layers.Dense(
        n_units, activation=tf.nn.relu,
        kernel_initializer=tf.initializers.RandomUniform(minval=-0.01,
                                                         maxval=0.01),
        bias_initializer=tf.initializers.constant(0.001)))

# finally add the output layer
layers.append(tf.keras.layers.Dense(
    10, kernel_initializer=tf.initializers.RandomUniform(minval=-0.01,
                                                         maxval=0.01)))

lr = 0.1
for step in range(2000):
    img_batch, lbl_batch = mnist.next_batch()

    # I hear adding random noise to inputs helps with generalization! 
    #commented below code to test the accuracy.
    img_batch = img_batch + tf.random.normal(tf.shape(img_batch), stddev=0)

    with tf.GradientTape() as tape:
        # here we just run all the layers in sequence via a for-loop
        out = img_batch
        for layer in layers:
            out = layer(out)
        xent = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=out, labels=lbl_batch))

    weights = [var for l in layers for var in l.trainable_variables]
    grads = tape.gradient(xent, weights)
    for grad, var in zip(grads, weights):
        var.assign_sub(lr*grad)

    if not step % 100:
        preds = tf.argmax(out, axis=1, output_type=tf.int32)
        acc = tf.reduce_mean(tf.cast(tf.equal(preds, lbl_batch), tf.float32))
        with train_writer.as_default():
          tf.summary.scalar("accuracy", acc, step=step)
          tf.summary.scalar("loss", xent, step=step)
          tf.summary.image("input", tf.reshape(img_batch, [-1, 28, 28, 1]), step=step)
          tf.summary.histogram("logits", out, step=step)
          tf.summary.histogram("weights", var, step=step)
          for var in weights:
            tf.summary.histogram(var.name,var,step=step)
          for i in range(0,len(weights)):
            tf.summary.histogram("Gradient-"+str(i),grads[i],step=step)
    
        print("Loss: {} Accuracy: {}".format(xent, acc))

Loss: 2.3025853633880615 Accuracy: 0.0859375
Loss: 2.3032422065734863 Accuracy: 0.1015625
Loss: 2.3012046813964844 Accuracy: 0.109375
Starting new epoch...
Loss: 2.2978227138519287 Accuracy: 0.125
Loss: 2.3035643100738525 Accuracy: 0.125
Starting new epoch...
Loss: 2.300755023956299 Accuracy: 0.125
Loss: 2.2992873191833496 Accuracy: 0.125
Loss: 2.2991855144500732 Accuracy: 0.1015625
Starting new epoch...
Loss: 2.2971301078796387 Accuracy: 0.13671875
Loss: 2.298671245574951 Accuracy: 0.10546875
Starting new epoch...
Loss: 2.300069808959961 Accuracy: 0.12109375
Loss: 2.3011765480041504 Accuracy: 0.10546875
Starting new epoch...
Loss: 2.3023598194122314 Accuracy: 0.10546875
Loss: 2.2998876571655273 Accuracy: 0.12890625
Loss: 2.300877571105957 Accuracy: 0.12109375
Starting new epoch...
Loss: 2.3037524223327637 Accuracy: 0.1015625
Loss: 2.299329996109009 Accuracy: 0.11328125
Starting new epoch...
Loss: 2.30256986618042 Accuracy: 0.09765625
Loss: 2.306338310241699 Accuracy: 0.10546875
Starti

In [33]:
out = mnist.test_data
for layer in layers:
    out = layer(out)
test_preds = tf.argmax(out, axis=1, output_type=tf.int32)
acc = tf.reduce_mean(tf.cast(tf.equal(test_preds, mnist.test_labels), tf.float32))
print("Final test accuracy: {}".format(acc))


Final test accuracy: 0.11349999904632568


In [34]:
# then load/run tensorboard
%load_ext tensorboard

In [35]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 149), started 1:15:04 ago. (Use '!kill 149' to kill it.)

<IPython.core.display.Javascript object>